In [13]:
!pip install jsonlines pandas datasets
!pip install zstandard



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 14.6 MB/s eta 0:00:00


In [14]:
import jsonlines
import itertools
import pandas as pd
from pprint import pprint

import datasets
from datasets import load_dataset

In [15]:
pretrained_dataset = load_dataset("EleutherAI/pile", split="train", streaming=True)

# pretrained_dataset = load_dataset("c4", "en", split="train", streaming=True)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for EleutherAI/pile contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/EleutherAI/pile
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [25]:
import json
python_dataset = [
  {
    "question": "What is Python?",
    "answer": "Python is a high-level, interpreted programming language known for its simplicity and readability."
  },
  {
    "question": "What are the key features of Python?",
    "answer": "Key features of Python include its dynamic typing, automatic memory management, and extensive standard library."
  },
  {
    "question": "How do you comment in Python?",
    "answer": "In Python, you can use the '#' symbol to create single-line comments, and ''' ... ''' or \"\"\" ... \"\"\" for multi-line comments."
  },
  {
    "question": "What is PEP 8?",
    "answer": "PEP 8 is the style guide for Python code, which recommends coding conventions to improve code readability."
  },
  {
    "question": "What are Python's data types?",
    "answer": "Python supports various data types including integers, floats, strings, lists, tuples, dictionaries, and sets."
  },
  {
    "question": "What is the difference between '==' and 'is' in Python?",
    "answer": "The '==' operator compares the values of two objects, while the 'is' operator checks if two variables refer to the same object in memory."
  },
  {
    "question": "How do you declare a variable in Python?",
    "answer": "Variables in Python are declared simply by assigning a value to a name. For example: 'x = 5'."
  },
  {
    "question": "What is a Python function?",
    "answer": "A function in Python is a block of reusable code that performs a specific task. It can take inputs, called parameters, and return outputs."
  },
  {
    "question": "How do you import a module in Python?",
    "answer": "You can import a module in Python using the 'import' keyword followed by the module name. For example: 'import math'."
  },
  {
    "question": "What is a list comprehension in Python?",
    "answer": "A list comprehension is a concise way of creating lists in Python by applying an expression to each item in an iterable."
  }
]


filename = "python_dataset.jsonl"

# Write the dataset to the JSONL file
with open(filename, "w") as f:
    for item in python_dataset:
        json.dump({"text": item}, f)  # Write each item as a JSON object
        f.write("\n")





In [26]:
n = 5
print("Python dataset:")
top_n = python_dataset[:n]  # Slice the dataset to get the top n elements
for i in top_n:
    print(i)

Python dataset:
{'question': 'What is Python?', 'answer': 'Python is a high-level, interpreted programming language known for its simplicity and readability.'}
{'question': 'What are the key features of Python?', 'answer': 'Key features of Python include its dynamic typing, automatic memory management, and extensive standard library.'}
{'question': 'How do you comment in Python?', 'answer': 'In Python, you can use the \'#\' symbol to create single-line comments, and \'\'\' ... \'\'\' or """ ... """ for multi-line comments.'}
{'question': 'What is PEP 8?', 'answer': 'PEP 8 is the style guide for Python code, which recommends coding conventions to improve code readability.'}
{'question': "What are Python's data types?", 'answer': 'Python supports various data types including integers, floats, strings, lists, tuples, dictionaries, and sets.'}


In [29]:
filename = "python_dataset.jsonl"
instruction_dataset_df = pd.read_json(filename, lines=True)
instruction_dataset_df

,text
0,"{'question': 'What is Python?', 'answer': 'Pyt..."
1,{'question': 'What are the key features of Pyt...
2,"{'question': 'How do you comment in Python?', ..."
3,"{'question': 'What is PEP 8?', 'answer': 'PEP ..."
4,"{'question': 'What are Python's data types?', ..."
5,{'question': 'What is the difference between '...
6,{'question': 'How do you declare a variable in...
7,"{'question': 'What is a Python function?', 'an..."
8,{'question': 'How do you import a module in Py...
9,{'question': 'What is a list comprehension in ...


In [30]:
examples = instruction_dataset_df.to_dict()
text = examples["question"][0] + examples["answer"][0]
text

KeyError: 'question'